In [ ]:
from unsloth import FastLanguageModel
from ollama import chat
from datasets import Dataset

In [ ]:
import re
import datetime
import itertools
import os
import pandas as pd
import numpy as np

from ast import literal_eval

# import src.inference as inference
# from src.utils import run_training

In [ ]:
# bert training

from deberta_fn import get_datasets, compute_metrics, tokenize_fn

task_name = 'stance_detection'
output_dir = f'./outputs/{task_name}/deberta_{task_name}'
model_name = 'microsoft/deberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
)

train_dataset, test_dataset = get_datasets(tokenizer, tokenize_fn, task_name)

# raise ValueError
training_args = TrainingArguments(
    output_dir=output_dir,
    eval_strategy="no",
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    gradient_accumulation_steps=1,
    num_train_epochs=5,
    learning_rate=3e-5,
    optim="adamw_torch_fused",
    weight_decay=0.01,
    logging_steps=1,
    report_to="tensorboard",
    save_strategy="epoch",
    # load_best_model_at_end=True,
    # eval_on_start=True,
    remove_unused_columns=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

# Test the model
test_results = trainer.evaluate(test_dataset)
print(test_results)
test_results
test_df = pd.Series(data=test_results)
test_df.to_csv(f'./test_res/{task_name}/deberta_res_{task_name}.csv', header=['score'])

In [ ]:
# Run Training, Test and saving 
# model, tokenizer = run('quality', do_training=True)

In [ ]:
# all_result = inference.inference_on_all_data(
#     model_for_task='corrected',
#     model_to_use='few-shot',
#     few_shot=True,
# )
# all_result

In [ ]:
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name=f'./merged_model/conf_della_4',
#     max_seq_length=2048,
#     dtype=None,
#     load_in_4bit=False,
#     fast_inference=True,
#     gpu_memory_utilization=0.6,
# )
# model.save_pretrained_gguf(
#     './gguf_model/merged_gguf',
#     tokenizer,
#     quantization_method = "q8_0", # choose quant method: q8_0
# )